## Bajar la base

In [ ]:
#Si no está descargado, es importante bajar el paquete pymrio
#!pip install pymrio   
import pymrio
from pathlib import Path

In [ ]:
oecd_storage = Path("/tmp/mrios/OECD")

In [ ]:
#elegir el año correspondiente
meta_download = pymrio.download_oecd(storage_folder=oecd_storage, years=[2018])

In [ ]:
oecd_path_year = pymrio.parse_oecd(path=oecd_storage, year=2018)

In [ ]:
oecd_path_year.factor_inputs.F.head()  #descargó la ICIO

In [ ]:
df=oecd_path_year
df?

In [ ]:
df_calculated= df.calc_all()  #de esta forma calcula la inversa de leontief y la matriz de coeficientes directos

## Manipular la base

Si se quiere elegir otros paises, seleccionar todas las celdas hasta el final, ir a **Editar** > **Encontrar y reemplazar** y cambiar la abreviatura del pais de tres letras por el correspondiente de la consulta. Luego correr el código.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
ARG_Y=df_calculated.Y.ARG  #matrix final demand
ARG_L=df_calculated.L.ARG  #inverse of leontief
ARG_Z=df_calculated.Z.ARG  #domestic demand

In [ ]:
sector_list=list(df_calculated.get_sectors())
region_list=list(df_calculated.get_regions())
region='ARG'

In [ ]:
len(sector_list) #cantidad de sectores
print('cantidad de sectores:', len(sector_list), '\ncantidad de paises:', len(region_list))

In [ ]:
m_identidad=np.identity(len(sector_list))  #matrix identidad
h_1=np.array([1 for v in range(len(sector_list))], dtype=int) #creo un vector horizontal de 1
v_1=np.array([[1] for v in range(len(sector_list))], dtype=int) #creo un vector vertical de 1

### Componentes directos e indirectos de la demanda doméstica

In [ ]:
leontief_ARG=ARG_L.loc[('ARG')] #leontief inverse from Spain
# should have 45 obs.
chequeo=list([x>1 for x in np.diagonal(leontief_ARG) ])
len(chequeo)

In [ ]:
#check that is correct, (all values in main diagonal should be greater than 1)
for x in chequeo:
    if x==False:
        print('error', chequeo.index(x)+1)  #error in the last obs corrARGond to social services, not a real problem
                                         # (Activities of households as employers; undifferentiated goods- and services-producing activities of households for own use)


### Exportaciones de ARG al resto del mundo

In [ ]:
del df_calculated.Y['ARG']
del df_calculated.Z['ARG']

#### Expo demandada por estructura productiva extranjera

In [ ]:
expo_internas=np.sum(df_calculated.Z.loc[('ARG')], axis=1)

In [ ]:
expo_internas.head()

#### Expo demanda por componentes de la demanda final ROW

In [ ]:
df_calculated.Y.loc[('ARG')]
expo_df=np.sum(df_calculated.Y.loc[('ARG')], axis=1)

In [ ]:
expo_df.head()

In [ ]:
X=expo_internas+expo_df
X.head()

### Demanda final de la producción nacional

In [ ]:
Fd=ARG_Y.loc[('ARG')] #tomando la demanda de produccion nacional, luego voy a tener que sumarle las impo directas, para obtener la demanda total
Fd.head()

In [ ]:
Fd['C']=Fd['HFCE']
Fd['I']=Fd['GFCF']

Fd.rename(columns={'GGFC':'G'}, inplace=True)

In [ ]:
Fd=Fd.loc[:,['C','G','I']]
Fd['X']=X
Fd.head()

### Coeficientes de importación

In [ ]:
total_pn=np.sum(ARG_Z, axis=0)   ##OJO, ACA PODRIA SER QUE LOS ESTOY CALCULANDO MAL, ES IMPO/DOM+IMPO O IMPO/DOM
#chequeo que lo este haciendo bien: total_pn[0] es el total de la primera columna -sector 01T02- de ARG.Z
#mientras genero un array con los datos de la primera columna de esta matriz y los sumo con np
col1=ARG_Z['01T02']
np.sum(col1)  #podria ser tamb print(np.sum(ARG_Z['01T02']))

print('comparacion:', total_pn[0],np.sum(col1)) # esta ok.

In [ ]:
total_dom=np.sum(ARG_Z.loc[('ARG')], axis=0)  #total de los requerimientos domesticos
total_imp=np.sum(ARG_Z.iloc[ARG_Z.index.get_level_values('region') != 'ARG'], axis=0)  #total de los requerimientos importados, le digo que del multiindex region me tome los que no son ARG

#### chequeo

In [ ]:
#chequeo que los totales de dom+impo = 
chequeo=total_dom+total_imp==total_pn
sum(chequeo==False) #hay 26 casos en que no dan igual, será por una diferencia grande?

In [ ]:
#generar un DF donde tenga todos los datos y sumar dom+imp manual y cotejar con pn.
lista=[total_dom, total_imp, total_pn]
base= pd.concat(lista, axis=1)
base.columns=['total_dom', 'total_imp', 'total_pn']

base['total_manual']=base.total_dom+base.total_imp  #sumado manual
base['dif']=round(base.total_pn-base.total_manual, 5) #la diferencia

In [ ]:
import matplotlib.pyplot as plot
plot.hist(x=base.dif)
plot.show()  # la diferencia esta muy cercana a cero.

In [ ]:
base['cm']=total_imp/total_pn


In [ ]:
vector_cm=np.array(base.cm)
vector_cm[-1]=0 #coeficiente de importacion de los servicios personales es igual a 0
vector_cm

In [ ]:
m=vector_cm*m_identidad
m

## Importaciones indirectas

[Bussiere et al (2013)](https://www.aeaweb.org/articles?id=10.1257/mac.5.3.118)

$M_{ind}=m(I-A)^{-1}F_d$

In [ ]:
Mind=np.matmul(np.matmul(m,leontief_ARG),Fd)
Mind.columns

Arreglar los componetes de demanda final

In [ ]:
#Mind['C']=Mind.HFCE+Mind.NPISH  # Households final consumption expenditure + Non-profit institutions serving households
#Mind['I']=Mind.GFCF+Mind.INVNT # - Gross fixed capital formation + Change in inventories and valuables

In [ ]:
#Mind.rename(columns={"GGFC": "G"}, inplace=True)  # General government final consumption and exports
#Mind=Mind.loc[:,['C','G','I']]

In [ ]:
Mind.set_index([df_calculated.get_sectors()]).head()

## Importaciones directas

[Bussiere et al (2013)](https://www.aeaweb.org/articles?id=10.1257/mac.5.3.118)

Son las filas de venta de sectores del ROW a los componentes de la demanda final doméstica

$M_{dir}=F_m$

In [ ]:
#ventas a la demanda final de ARG por los sectores de otros paises.
Mdir=ARG_Y.iloc[ARG_Y.index.get_level_values('region') != 'ARG'].groupby(level=1).sum() 
Mdir['C']=Mdir['HFCE']
Mdir['I']=Mdir['GFCF'] 

Mdir.rename(columns={'GGFC':'G'}, inplace=True)

In [ ]:
Mdir=Mdir.loc[:,['C','G','I']] #es raro que tenga venta directa de otros paises a la demanda externa (X), mejor sacar re-exportaciones
Mdir.head()

## Intensidad importadora de los componentes de demanda

[Bussiere et al (2013)](https://www.aeaweb.org/articles?id=10.1257/mac.5.3.118)

$w_k=\frac{uMind_k+uMdir_k}{uFm_k+uFd_k}$  

donde `u` un vector de unos

In [ ]:
demanda=['C','G','I']
w=[]
for x in demanda:
 w.append((np.matmul(h_1,Mind[x])+np.matmul(h_1,Mdir[x]))/(np.matmul(h_1,Fd[x])+np.matmul(h_1,Mdir[x])))

In [ ]:
#exportaciones las pongo aparte porque no aparecen en Mdir
w.append((np.matmul(h_1,Mind['X']))/(np.matmul(h_1,Fd['X'])))   

In [ ]:
w

In [ ]:
wdir=[]
for x in demanda:
 wdir.append((np.matmul(h_1,Mdir[x]))/(np.matmul(h_1,Fd[x])+np.matmul(h_1,Mdir[x])))


In [ ]:
wdir

In [ ]:
wind=[]
for x in demanda:
 wind.append((np.matmul(h_1,Mind[x]))/(np.matmul(h_1,Fd[x])+np.matmul(h_1,Mdir[x])))

wind.append((np.matmul(h_1,Mind['X']))/(np.matmul(h_1,Fd['X'])))  

In [ ]:
wind

In [ ]:
demanda=['C','G','I','X']
df1 = pd.DataFrame(wind,index=demanda)
df1.rename({0: 'wind'}, axis=1, inplace=True)

In [ ]:
demanda=['C','G','I']
df2 = pd.DataFrame(wdir,index=demanda)
df2.rename({0: 'wdir'}, axis=1, inplace=True)

In [ ]:
demanda=pd.Index(['C','G','I','X'])
df3 = pd.DataFrame(w,index=demanda)
df3.rename({0: 'wgt'}, axis=1, inplace=True)

In [ ]:
import os
os.getcwd()
os.chdir("...your path...")

In [ ]:
df=df3.join(df1.join(df2))
df.wdir[df['wdir'].isna()==True]=0
df

In [ ]:
df.to_excel("wgt_ICIO18.xlsx")